In [1]:
# Extraction depuis CSV
import csv
tweets_data = []  # On initialise une liste vide pour stocker les tweets et leurs labels
with open("balanced_tweets_200k.csv", "r", encoding="latin1") as f:
    reader = csv.reader(f)
    for row in reader:
        if len(row) >= 6:
            raw_label = row[0]
            tweet = row[5]

            # Conversion du label brut en label textuel (positif/négatif)
            if raw_label == "0":
                label = "négatif"
            elif raw_label == "4":
                label = "positif"
            else:
                continue
            tweets_data.append((tweet, label))

# Extraction des colonnes
raw_tweets = [t[0] for t in tweets_data]
labels = [t[1] for t in tweets_data]
print(raw_tweets[:5])
print(labels[:5])

["Looks like the sun finally located Trondheim ;-) hope summer's on it's way ", "A long weekend begins. The sun is shining and I'm happy ! Exams soon ", 'to the beach we go! hope it stays nice... ', '@JBFutureboy I missed it  busted need to do a reunion tour. That would make my year. No joke.', "Why I can't change my background image?? "]
['positif', 'négatif', 'positif', 'négatif', 'négatif']


In [2]:
import re #pour utiliser expressions régulières
from pathlib import Path #pour manipuler les fichiers

import nltk #notre bibliothèque de traitement de texte documenté
from nltk.corpus import stopwords
from nltk.tokenize import TreebankWordTokenizer #tokenizer : transforme les phrases en liste de mots

# Télécharger les ressources NLTK
nltk.download("punkt", quiet=True)
nltk.download("stopwords", quiet=True)

# Initialisation du tokenizer (plus robuste que word_tokenize)
tokenizer = TreebankWordTokenizer()

# Préparation des regex
# on supprime les artefacts typiques des tweets encodés de manière étrange
regex_artifacts_sources = re.compile(
    r'\|'
    r'\[Saut de retour à la ligne\]|'
    r'(?:<ed><U\+[0-9A-F]{4}><U\+[0-9A-F]{4}>)+|'
    r'<U\+[0-9A-F]{4,6}>|'
    r'<ed>'
)

# on supprime les noms d'utilisateurs mentionnés
regex_usernames = re.compile(r'@\w+')

# on supprime les URL (http ou https)
regex_urls = re.compile(r'https?://\S+')

# on supprime les hashtags uniquement
regex_hashtags = re.compile(r'#\w+')

# on supprime les contractions comme l', j', c', etc.
regex_apostrophes = re.compile(r"\b\w+'")

# on supprime les guillemets doubles
regex_quotes = re.compile(r'"')

# on supprime la ponctuation générale
regex_punctuation = re.compile(r'[.,;:!?()\[\]{}\\/|`~^<>«»=]')

# Traitement des fichiers

# Crée le dossier de sortie s’il n’existe pas
#output_dir = Path("CorpusRandomCleaned")
#output_dir.mkdir(parents=True, exist_ok=True)

# Définition du chemin de sortie pour les tweets nettoyés
#dest_file = output_dir / f"cleaned_tweets{i}.txt"

cleaned_tweets = []

# Choix de la langue pour les stopwords
langue = "english"
stop_words = set(stopwords.words(langue))

for tweet_text in raw_tweets:

    # Nettoyage du texte
    # Supprime les artefacts liés à l'encodage
    t = regex_artifacts_sources.sub('', tweet_text)
    # Corrige les doubles guillemets ("" → ")
    t = t.replace('""', '"')
    # Supprime les noms d’utilisateurs
    t = regex_usernames.sub('', t)
    # Supprime les liens
    t = regex_urls.sub('', t)
    # Supprime les hashtags
    t = regex_hashtags.sub('', t)
    # Supprime les contractions du type l', j', etc.
    t = regex_apostrophes.sub('', t)
    # Supprime tous les guillemets
    t = regex_quotes.sub('', t)
    # Supprime les apostrophes restantes
    t = t.replace("'", "")
    # Supprime la ponctuation
    t = regex_punctuation.sub(' ', t)
    # Met en minuscule
    t = t.lower()
    # Supprime les espaces multiples
    t = re.sub(r'\s{2,}', ' ', t).strip()

    # Tokenisation
    tokens = tokenizer.tokenize(t)

    # Conservation de RT en majuscule même après passage en minuscule
    filtered_tokens = [
        word.upper() if word.lower() == 'rt' else word
        for word in tokens
        if word.lower() not in stop_words or word.lower() == 'rt'
    ]

    # On garde le tweet sous forme de liste de mots
    cleaned_tweets.append(f"{filtered_tokens}")

# Sauvegarde du fichier nettoyé
#with open(dest_file, "w", encoding="utf-8") as f:
#    f.write("\n".join(cleaned_tweets))

In [13]:
#Importation des fonctions et librairies

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, train_test_split
import numpy as np
import pandas as pd

# Externalisation des entraînements de modèles et inférences sur GPU et non CPU

%load_ext cuml.accel

# Calcule le score TF-IDF automatiquement

tfidf = TfidfVectorizer(
    ngram_range=(1, 2),
    max_features=15000,      # Limite le vocabulaire
    min_df=5,               # Ignore les mots présents dans <5 documents
    max_df=0.7,             # Ignore les mots présents dans >70% des documents
    sublinear_tf=True,
    analyzer='word',
    stop_words='english'    # Utilise la liste de stop words intégrée
)

The cuml.accel extension is already loaded. To reload it, use:
  %reload_ext cuml.accel


In [21]:
# On appelle la fonction TF-IDF sur nos données

data = tfidf.fit_transform(cleaned_tweets)
#data = np.array(cleaned_tweets)
X = data
Y = [1 if label == "positif" else -1 for label in labels]
n = data.shape[0] # nombre de données (utilisé pour n'entraîner que sur une partie pour des petits tests)

X_train, X_test, y_train, y_test = train_test_split(X[:n], Y[:n], test_size=0.2, random_state=42) # séparation des datasets d'entraînement + validation (cross-validation) et de test, avec du déterminisme (seed fixée)

Régression logistique

In [ ]:
import numpy as np
from sklearn.model_selection import GridSearchCV

model = LogisticRegression(
    penalty='l2',
    C=1.0,               # Régularisation
    #l1_ratio=0.4,  # Ratio de régularisation L1
    class_weight='balanced',  # Équilibrage des classes
    solver='liblinear',  # Algorithme de résolution
    max_iter=1000,       # Nombre maximal d'itérations
    random_state=42      # Pour la reproductibilité
)

# Convertit y_train et y_test en NumPy arrays avec un dtype spécifique pour éviter les erreurs et map -1 à 0
y_train_np = np.array([(1 if y == 1 else 0) for y in y_train], dtype=np.int32)
y_test_np = np.array([(1 if y == 1 else 0) for y in y_test], dtype=np.int32)

# On définit la grille de tests d'hyperparamètres pour la cross-validation
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2']
}
grid = GridSearchCV(model, param_grid, cv=10, scoring='accuracy', verbose=1) #cross validation
grid.fit(X_train, y_train_np) #entraînement
print(grid.best_params_)
best_model = grid.best_estimator_ #on choisit le meilleur
best_model.fit(X_train, y_train_np) #on le ré-entraîne sur tout
y_pred = best_model.predict(X_test)
accuracy = best_model.score(X_test, y_test_np)
print("Test set accuracy: {:.2f}%".format((accuracy) * 100))

[2025-06-05 08:15:19.304] [CUML] [info] Unused keyword parameter: random_state during cuML estimator initialization
[2025-06-05 08:15:19.325] [CUML] [info] Unused keyword parameter: dual during cuML estimator initialization
[2025-06-05 08:15:19.325] [CUML] [info] Unused keyword parameter: intercept_scaling during cuML estimator initialization
[2025-06-05 08:15:19.325] [CUML] [info] Unused keyword parameter: multi_class during cuML estimator initialization
[2025-06-05 08:15:19.325] [CUML] [info] Unused keyword parameter: n_jobs during cuML estimator initialization
[2025-06-05 08:15:19.325] [CUML] [info] Unused keyword parameter: random_state during cuML estimator initialization
[2025-06-05 08:15:19.325] [CUML] [info] Unused keyword parameter: warm_start during cuML estimator initialization
Fitting 10 folds for each of 12 candidates, totalling 120 fits
[2025-06-05 08:15:19.343] [CUML] [info] Unused keyword parameter: dual during cuML estimator initialization
[2025-06-05 08:15:19.343] [CU

Random Forest

In [ ]:
rf_model = RandomForestClassifier(
    n_estimators=100,  # Nombre d'arbres
    criterion='gini',    # Critère de split (gini ou entropy)
    max_depth=None,      # Profondeur maximale de l'arbre
    min_samples_split=2, # Nombre minimum d'échantillons requis pour spliter un nœud interne
    min_samples_leaf=1,  # Nombre minimum d'échantillons requis à un nœud feuille
    random_state=42      # Pour la reproductibilité
)

# On convertit les "sparse matrices" en "dense arrays" pour ne pas avoir d'erreurs avec cuML
X_train_dense = X_train.toarray()
X_test_dense = X_test.toarray()

# On convertit y_train et y_test en NumPy arrays avec le bon dtype
y_train_np = np.array(y_train, dtype=np.float32)
y_test_np = np.array(y_test, dtype=np.float32)

rf_model.fit(X_train_dense, y_train_np) #entraînement

# NB : on peut refaire de la cross-validation ici aussi, typiquement sur le nombre d'arbres ou le critère

y_pred_rf = rf_model.predict(X_test_dense)

accuracy_rf = rf_model.score(X_test_dense, y_test_np)
print(f"Random Forest Accuracy: {accuracy_rf:.2%}")

/usr/local/lib/python3.11/dist-packages/cupy/_creation/from_data.py:88: PerformanceWarning: Using synchronous transfer as pinned memory (19200000000 bytes) could not be allocated. This generally occurs because of insufficient host memory. The original error was: cudaErrorMemoryAllocation: out of memory
  return _core.array(a, dtype, False, order, blocking=blocking)


Mixture of Experts

In [ ]:
# Definition des modèles experts - on en prend des nouveaux mais on pourrait en récupérer des pré-entraînés
expert1 = LogisticRegression(
    penalty='l2',
    C=1.0,
    class_weight='balanced',
    solver='liblinear',
    max_iter=1000,
    random_state=42
)

expert2 = RandomForestClassifier(
    n_estimators=100,
    criterion='gini',
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42
)

expert3 = SVC(probability=True, random_state=42)


# On crée le modèles qui vote (Mixture of Experts)
# 'voting'='hard' => majority voting, 'voting'='soft' => predicted probabilities
moe_model = VotingClassifier(
    estimators=[('lr', expert1), ('rf', expert2), ('svm', expert3)],
    voting='hard'
)

# Entraînement (encore et toujours)
moe_model.fit(X_train, y_train)

# Prédictions
y_pred_moe = moe_model.predict(X_test)

accuracy_moe = moe_model.score(X_test, y_test)
print(f"Mixture of Experts Accuracy: {accuracy_moe:.2%}")

Réseau de neurones (élémentaire, pour un plus avancer aller sur RNplusplus (ou les autres RN... déjà))

In [ ]:
#!pip install tensorflow keras

# Encore un petit paquet d'imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# On construit l'architecture du réseau et les fonctions d'activation
input_layer = Input(shape=(X_train.shape[1],))
dense_layer_1 = Dense(128, activation='relu')(input_layer)
dropout_layer_1 = Dropout(0.5)(dense_layer_1)
dense_layer_2 = Dense(64, activation='relu')(dropout_layer_1)
dropout_layer_2 = Dropout(0.5)(dense_layer_2)
output_layer = Dense(1, activation='sigmoid')(dense_layer_2) # Using sigmoid for binary classification

nn_model = Model(inputs=input_layer, outputs=output_layer)

# On génère une instance de ce modèle
nn_model.compile(optimizer=Adam(learning_rate=0.001),
                 loss='binary_crossentropy', # Using binary crossentropy for binary classification
                 metrics=['accuracy'])

# On doit convertir les labels en 0 et 1 pour binary crossentropy
y_train_nn = [(1 if y == 1 else 0) for y in y_train]
y_test_nn = [(1 if y == 1 else 0) for y in y_test]

# Entraînement
# On convertit les "sparse matrix" en "dense array" pour le GPU
X_train_dense = X_train.toarray()
X_test_dense = X_test.toarray()

# Conversion des NumPy arrays en TensorFlow Tensors (changement de librairie oblige)
X_train_tensor = tf.convert_to_tensor(X_train_dense, dtype=tf.float32)
X_test_tensor = tf.convert_to_tensor(X_test_dense, dtype=tf.float32)
y_train_tensor = tf.convert_to_tensor(y_train_nn, dtype=tf.float32)
y_test_tensor = tf.convert_to_tensor(y_test_nn, dtype=tf.float32)


nn_model.fit(X_train_tensor, y_train_tensor, epochs=50, batch_size=10, verbose=0) #Le vrai entraînement ! On peut bouger le nombre d'epochs ou la batch size

# On calcule la précision
loss, accuracy_nn = nn_model.evaluate(X_test_tensor, y_test_tensor, verbose=0)
print(f"Neural Network Accuracy: {accuracy_nn:.2%}")

# On regarde ce qu'il se passe sur les labels prédits et réels pour comparer "à la main"
# Comme la sortie est en probas, on convertit en -1/1
y_pred_nn_prob = nn_model.predict(X_test_tensor)
y_pred_nn = [(1 if prob > 0.5 else -1) for prob in y_pred_nn_prob]

print("Neural Network Predictions:", y_pred_nn)
print("Actual Test Labels:", y_test)

XGBoost (variante RF)

In [ ]:
#!pip install xgboost

import xgboost as xgb

# XGBoost a besoin de DMatrix en interne
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# On définit les hyperparamètres de XGBoost

y_train_xgb = [(1 if y == 1 else 0) for y in y_train] # pour utiliser la binary classification
y_test_xgb = [(1 if y == 1 else 0) for y in y_test]

dtrain = xgb.DMatrix(X_train, label=y_train_xgb)
dtest = xgb.DMatrix(X_test, label=y_test_xgb)


params = {
    'objective': 'binary:logistic',  # Binary classification avec logistic regression
    'eval_metric': 'logloss',        # Métrique
    'eta': 0.5,                      # Learning rate
    'max_depth': 3,
    'subsample': 0.8,                # Pourcentage d'entraînement (80-20%)
    'colsample_bytree': 0.8,         # Pourcentage de colonnes utilisées pour un arbre
    'seed': 42                       # Reproducibilité de l'aléatoire
}

# Entraînement de XGBoost
num_rounds = 100  # Nombre de boosting rounds
watchlist = [(dtrain, 'train'), (dtest, 'eval')] # Permet de surveiller l'évolution de la performance

xgb_model = xgb.train(params, dtrain, num_rounds, evals=watchlist, early_stopping_rounds=10, verbose_eval=False) #Entraînement !

# Prédictions
# XGBoost renvoie des probas avec binary:logistic
y_pred_xgb_prob = xgb_model.predict(dtest)

# On convertit ces probabilités en labels (0 or 1)
y_pred_xgb = [1 if prob > 0.5 else 0 for prob in y_pred_xgb_prob]

# On reconvertit au format -1, 1 pour comparer avec y_test
y_pred_xgb_original = [1 if pred == 1 else -1 for pred in y_pred_xgb]


# On évalue les performances
from sklearn.metrics import accuracy_score
accuracy_xgb = accuracy_score(y_test_xgb, y_pred_xgb)

print(f"XGBoost Accuracy: {accuracy_xgb:.2%}")

print("XGBoost Predictions (0/1):", y_pred_xgb)
print("XGBoost Predictions (-1/1):", y_pred_xgb_original)
print("Actual Test Labels (-1/1):", y_test)
print("Actual Test Labels (0/1):", y_test_xgb)


k Nearest Neighbors

In [15]:
from sklearn.neighbors import BallTree
print(sorted(BallTree.valid_metrics))
# Juste tests pour les métriques ci-dessous (sorte de cross-validation manuelle pour éviter la surcharge de mémoire)

['braycurtis', 'canberra', 'chebyshev', 'cityblock', 'dice', 'euclidean', 'hamming', 'haversine', 'infinity', 'jaccard', 'l1', 'l2', 'mahalanobis', 'manhattan', 'minkowski', 'p', 'pyfunc', 'rogerstanimoto', 'russellrao', 'seuclidean', 'sokalmichener', 'sokalsneath']


In [23]:
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

knn_model = KNeighborsClassifier(
    n_neighbors=3,  # nombre de voisins à considérer
    weights='distance',
    algorithm='ball_tree', # Algorithme utilisé pour trouver les voisins
    leaf_size=30,     # argument de l'algo
    #p=2,              # Ppuissance pour la métrique 'minkovski'
    metric='l1', # Distance
)

# Conversion de y_train et y_test en NumPy arrays
y_train_np = np.array(y_train, dtype=np.int32)
y_test_np = np.array(y_test, dtype=np.int32)

knn_model.fit(X_train, y_train_np) # Entraînement

y_pred_knn = knn_model.predict(X_test)

accuracy_knn = knn_model.score(X_test, y_test_np)
#print(f"kNN Accuracy: {accuracy_knn:.2%}")
#print("kNN Predictions:", y_pred_knn)
#print("Actual Test Labels:", y_test_np)
# Cross validation
param_grid = {'n_neighbors': list(range(1, 5))}
grid = GridSearchCV(knn_model, param_grid, cv=10, scoring='accuracy', verbose=1)
grid.fit(X_train, y_train_np)
print(grid.best_params_)
best_knn = grid.best_estimator_
y_pred = best_knn.predict(X_test)
print("Test set accuracy: {:.2f}%".format(accuracy_score(y_test_np, y_pred) * 100))

[2025-06-05 21:29:21.218] [CUML] [info] Unused keyword parameter: leaf_size during cuML estimator initialization
[2025-06-05 21:29:45.687] [CUML] [info] Unused keyword parameter: leaf_size during cuML estimator initialization
[2025-06-05 21:29:45.687] [CUML] [info] Unused keyword parameter: n_jobs during cuML estimator initialization
Fitting 10 folds for each of 4 candidates, totalling 40 fits
[2025-06-05 21:29:45.707] [CUML] [info] Unused keyword parameter: leaf_size during cuML estimator initialization
[2025-06-05 21:29:45.707] [CUML] [info] Unused keyword parameter: n_jobs during cuML estimator initialization
[2025-06-05 21:29:49.816] [CUML] [info] Unused keyword parameter: leaf_size during cuML estimator initialization
[2025-06-05 21:29:49.816] [CUML] [info] Unused keyword parameter: n_jobs during cuML estimator initialization
[2025-06-05 21:29:53.838] [CUML] [info] Unused keyword parameter: leaf_size during cuML estimator initialization
[2025-06-05 21:29:53.838] [CUML] [info] Unus

In [ ]:
# Support Vector Machine (SVM)
svm_model = SVC(
    C=1.0,                 # Régularisation (plus C est grand, moins la régularisation est forte)
    kernel='linear',       # Type de noyau ('linear', 'poly', 'rbf', 'sigmoid')
    gamma='scale',         # Coefficient de noyau pour 'rbf', 'poly', 'sigmoid'
    probability=True,      # Permet de calculer les probabilités de classe
    random_state=42        # Pour la reproductibilité
)

svm_model.fit(X_train, y_train) # Entraînement

y_pred_svm = svm_model.predict(X_test)

accuracy_svm = svm_model.score(X_test, y_test)
print(f"SVM Accuracy: {accuracy_svm:.2%}")
print("SVM Predictions:", y_pred_svm)
print("Actual Test Labels:", y_test)